### Imports

In [23]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Load the beta-VAE module
import bvae_model as bvae
import spectrum_preprocessing as sp

### Load raw spectra

In [24]:
#your path

path = "C:/Users/Zach/Documents/betaVAE_Training/training_data/SMP65#010 70d 820um.csv"

In [25]:
filename = path.split("/")[-1]
filename

'SMP65#010 70d 820um.csv'

In [ ]:
df = pd.read_csv(path, skiprows=[1,2])  # Skip metadata rows if present

if df.isnull().values.any():
    df = df.fillna(method='ffill').fillna(method='bfill')

print(f"Dataframe shape: {df.shape}")

C:\Users\Zach\AppData\Local\Temp\ipykernel_4052\4141153114.py:1: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

Dataframe shape: (6146, 2100)


In [27]:
df

,map_x,map_y,Sample Name,1981.7 - 2095.8,3997.9328286151754,3996.348861567388,3994.7648945196006,3993.1809274718134,3991.5969604240263,3990.0129933762387,...,693.7775669308426,692.1935998830554,690.6096328352678,689.0256657874806,687.4416987396935,685.8577316919059,684.2737646441187,682.6897975963311,681.105830548544,679.5218635007568
0,continuous,continuous,discrete,continuous,continuous,continuous,continuous,continuous,continuous,continuous,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous
1,meta,meta,meta,meta,continuous,continuous,continuous,continuous,continuous,continuous,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous
2,40253,23554.2,70d 820um,3.54994,0.0372556,0.0365067,0.0358452,0.0358513,0.0362444,0.0363316,...,-0.275936,-0.182681,-0.153185,-0.198179,-0.253059,-0.2721,-0.305597,-0.225356,-0.19798,-0.18254
3,40258.4,23554.2,70d 820um,3.43524,0.0390324,0.0386142,0.0386733,0.0387285,0.038532,0.0385616,...,-0.037869,-0.0837124,-0.10823,-0.100065,-0.0875998,-0.0802599,-0.0563268,-0.0145323,0.011343,0.00798068
4,40263.9,23554.2,70d 820um,3.5187,0.0358065,0.0360026,0.0373343,0.0384012,0.0379233,0.0365976,...,0.0505495,0.0212753,0.0227568,0.0404046,0.060266,0.0641239,0.0375718,-0.00614634,-0.0383616,-0.0226855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,40573.6,24069.3,70d 820um,3.59903,0.034905,0.034986,0.034399,0.03377,0.033633,0.033888,...,-0.07935,-0.102773,-0.14373,-0.178223,-0.172435,-0.148927,-0.12402,-0.114703,-0.117844,-0.113142
6142,40579.0,24069.3,70d 820um,3.5741,0.040409,0.040744,0.041003,0.04134,0.04163,0.041937,...,-0.15561,-0.128756,-0.134918,-0.184649,-0.224974,-0.248244,-0.267434,-0.249948,-0.264085,-0.409678
6143,40584.4,24069.3,70d 820um,3.52141,0.032184,0.031725,0.031192,0.031232,0.032178,0.033412,...,-0.080312,-0.102963,-0.13893,-0.220757,-0.347931,-0.236641,-0.222966,-0.235755,-0.139983,-0.080641
6144,40589.9,24069.3,70d 820um,3.43472,0.033654,0.033139,0.032943,0.033346,0.03389,0.034022,...,-0.293151,-0.254141,-0.223883,-0.199118,-0.169626,-0.164825,-0.165135,-0.136336,-0.105745,-0.106823


# Sort the wavenumber columns and data and put in a dataframe
- necessary for input to pipeline below

In [28]:
wavenumber = df.columns.values[4:].astype(float)
wavenumber.sort()
wavenumber = wavenumber.astype(str)
spectra_df = df[wavenumber].copy()
np.dtype(spectra_df.columns.values.astype(float)[0])

dtype('float64')

### Run the pipeline
The `bvae.pipeline()` returns the interpolated wavenumber array; the interpolated absorbance array; the beta-model encodings L1 L2 and L3; the reconstruction MSE; and the reconstructed spectrum.

In [29]:
L1_list=[]
L2_list=[]
L3_list=[]
MSE_list=[]
wavenumber_list=[]
absorbance_list=[]
reconstructed_list=[]
for i in range(spectra_df.shape[0]):    
    f = spectra_df.columns.values[:].astype('float32')
    a = spectra_df.iloc[i,:].values
    try:
        wavenumber, absorbance, encodings, mse, reconstructed = bvae.bvae_pipeline(f, a)
        L1 = encodings[0]
        L2 = encodings[1]
        L3 = encodings[2]
    except Exception as e:
        print(e)
        L1 = 0
        L2 = 0
        L3 = 0
        mse = 100
        reconstructed = np.zeros(len(wavenumber))
        
    L1_list.append(L1)
    L2_list.append(L2)
    L3_list.append(L3)
    MSE_list.append(mse)
    absorbance_list.append(absorbance)
    reconstructed_list.append(reconstructed)
    
    #print(encodings)
    #print(mse)
    #plt.plot(absorbance,c='b')
    #plt.plot(reconstructed,c='r')
    #plt.show()

could not convert string to float: 'continuous'
could not convert string to float: 'continuous'
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!
airpls: max iteration reached!


KeyboardInterrupt: 

# Save results to dataframe

In [ ]:
hyperspectrum =pd.DataFrame(absorbance_list)
hyperspectrum.columns = wavenumber
hyperspectrum['map_x'] = df['map_x'].values
hyperspectrum['map_y'] = df['map_y'].values

In [ ]:
reconstructed =pd.DataFrame(reconstructed_list)
reconstructed.columns = wavenumber
reconstructed['map_x'] = df['map_x'].values
reconstructed['map_y'] = df['map_y'].values

ValueError: Length mismatch: Expected axis has 2096 elements, new values have 1063 elements

In [ ]:
bvae_df =pd.DataFrame()
bvae_df['map_x'] = df['map_x'].values
bvae_df['map_y'] = df['map_y'].values
bvae_df['L1'] = L1_list
bvae_df['L2'] = L2_list
bvae_df['L3'] = L3_list
bvae_df['MSE'] = MSE_list

# Save to csv files for viewing in Quasar

In [ ]:

path_to_save = 'C:/Users/Zach/Documents/BVAE_processed/'
os.makedirs(path_to_save, exist_ok=True)
bvae_df.to_csv(path_to_save +'bvae_'+filename)

In [ ]:
path_to_save = "C:/Users/Zach/Documents/Hyperspectra/"
os.makedirs(path_to_save, exist_ok=True)
file=os.path.join(path_to_save + 'hyperspectrum_'+filename)
hyperspectrum.to_csv(file, index=False)

In [ ]:
path_to_save = "C:/Users/Zach/Documents/Reconstructions/"
os.makedirs(path_to_save, exist_ok=True)
reconstructed.to_csv(path_to_save +'reconstructed_'+filename, index=False)